In [9]:
from PIL import Image
import numpy as np
import pandas as pd

# Load the image
image = Image.open('/content/fruits.jpg')

# Convert the image to a numpy array if you need to work with numerical data
image_array = np.array(image)

In [2]:
"""*****************************************************************************************
IIIT Delhi License
Copyright (c) 2023 Supratim Shit
*****************************************************************************************"""
from sklearn.datasets import make_blobs
from sklearn.datasets import fetch_kddcup99
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
import wkpp as wkpp
import numpy as np
import random


data = image_array
data=  make_blobs(n_samples=100,n_features=17)
dataset = fetch_kddcup99()								# Fetch kddcup99
data = dataset.data										# Load data
data = np.delete(data,[0,1,2,3],1) 						# Preprocess
data = data.astype(float)								# Preprocess
data = StandardScaler().fit_transform(data)				# Preprocess

n = np.size(data,0)										# Number of points in the dataset
d = np.size(data,1)										# Number of dimension/features in the dataset.
k = 17													# Number of clusters (say k = 17)
Sample_size = 100										# Desired coreset size (say m = 100)


def D2(data, k):
    n_samples, n_features = data.shape
    B = np.zeros((k, n_features))
    # Randomly choose the first center
    B[0] = data[np.random.randint(n_samples)]

    for i in range(1, k):
        # Calculate the squared Euclidean distances from each data point to the nearest already chosen center
        if i == 1:
            distances = np.sum((data - B[0])**2, axis=1)
        else:
            distances = np.min(cdist(data, B[:i])**2, axis=1)

        # Normalize distances to get probabilities
        total_distance = np.sum(distances)
        probabilities = distances / total_distance

        # Ensure 'a' is an array of indices from which to sample, and 'p' is the probabilities array
        selected_idx = np.random.choice(np.arange(n_samples), p=probabilities)
        B[i] = data[selected_idx]

    return B 										# Returns B from Algo-1.

centers = D2(data,k)									# Call D2-Sampling (D2())

def Sampling(data, k, centers, Sample_size):
    # Assume the sensitivities are calculated based on the nearest center distances
    distances = np.min(cdist(data, centers) ** 2, axis=1)
    total_distance = np.sum(distances)
    sensitivities = distances / total_distance
    sampling_probabilities = sensitivities / np.sum(sensitivities)
    sample_indices = np.random.choice(data.shape[0], size=Sample_size, replace=True, p=sampling_probabilities)
    coreset = data[sample_indices]
    weight = np.ones(Sample_size) * (1 / sampling_probabilities[sample_indices]) * (1 / Sample_size)

    return coreset, weight 								# Return coreset points and its weights.

coreset, weight = Sampling(data,k,centers,Sample_size)	# Call coreset construction algorithm (Sampling())

#---Running KMean Clustering---#
fkmeans = KMeans(n_clusters=k,init='k-means++')
fkmeans.fit_predict(data)

#----Practical Coresets performance----#
Coreset_centers, _ = wkpp.kmeans_plusplus_w(coreset, k, w=weight, n_local_trials=100)						# Run weighted kMeans++ on coreset points
wt_kmeansclus = KMeans(n_clusters=k, init=Coreset_centers, max_iter=10).fit(coreset,sample_weight = weight)	# Run weighted KMeans on the coreset, using the inital centers from the above line.
Coreset_centers = wt_kmeansclus.cluster_centers_															# Compute cluster centers
coreset_cost = np.sum(np.min(cdist(data,Coreset_centers)**2,axis=1))										# Compute clustering cost from the above centers
reative_error_practicalCoreset = abs(coreset_cost - fkmeans.inertia_)/fkmeans.inertia_						# Computing relative error from practical coreset, here fkmeans.inertia_ is the optimal cost on the complete data.

#-----Uniform Sampling based Coreset-----#
tmp = np.random.choice(range(n),size=Sample_size,replace=False)
sample = data[tmp][:]																						# Uniform sampling
sweight = n*np.ones(Sample_size)/Sample_size 																# Maintain appropriate weight
sweight = sweight/np.sum(sweight)																			# Normalize weight to define a distribution

#-----Uniform Samling based Coreset performance-----#
wt_kmeansclus = KMeans(n_clusters=k, init='k-means++', max_iter=10).fit(sample,sample_weight = sweight)		# Run KMeans on the random coreset
Uniform_centers = wt_kmeansclus.cluster_centers_															# Compute cluster centers
uniform_cost = np.sum(np.min(cdist(data,Uniform_centers)**2,axis=1))										# Compute clustering cost from the above centers
reative_error_unifromCoreset = abs(uniform_cost - fkmeans.inertia_)/fkmeans.inertia_						# Computing relative error from random coreset, here fkmeans.inertia_ is the optimal cost on the full data.


print("Relative error from Practical Coreset is",reative_error_practicalCoreset)
print("Relative error from Uniformly random Coreset is",reative_error_unifromCoreset)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1362: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Relative error from Practical Coreset is 3.8148776731353693
Relative error from Uniformly random Coreset is 1.6206192970904223


In [3]:
pip install opencv-python

In [4]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Step 1: Cluster the image data using KMeans
fkmeans = KMeans(n_clusters=k, init='k-means++')
cluster_labels = fkmeans.fit_predict(image_array.reshape(-1, 3))  # Reshape image for KMeans

# Step 2: Assign each pixel to its nearest cluster center
cluster_centers = fkmeans.cluster_centers_
segmented_image_data = cluster_centers[cluster_labels]

# Step 3: Reshape the modified data back into the image shape
segmented_image = segmented_image_data.reshape(image_array.shape)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [5]:
image1=Image.fromarray(segmented_image_data.astype(np.uint8))
scaled_segmented_image_data=np.clip(image1,0,255).astype(np.uint8)

In [6]:
import numpy as np
from PIL import Image

# Convert the data type of segmented_image_data to uint8
segmented_image_data_uint8 = segmented_image_data.astype(np.uint8)

# Reshape the segmented image data to match the shape of the original image array
segmented_image_data_reshaped = segmented_image_data_uint8.reshape(image_array.shape)

# Convert the numpy array to an image
image = Image.fromarray(segmented_image_data_reshaped)

# Check the mode of the image
print("Image mode:", image.mode)

# If the mode is not compatible with PNG, convert it to RGB or RGBA
if image.mode != 'RGB' and image.mode != 'RGBA':
    image = image.convert('RGB')  # or 'RGBA' if you have an alpha channel

# Save the image to a file
image.save("fruits_segmented.png")

# Optionally, display the image
image.show()


Image mode: RGB
